In [80]:
import pandas as pd 
from datetime import datetime
import matplotlib.pyplot as plt
import utils
import numpy as np
import importlib 
from tqdm import tqdm

importlib.reload(utils)


<module 'utils' from 'c:\\Users\\UFMG\\Periodos\\Periodo_XIV\\TCC\\Projeto\\Códigos\\tcc_ovitraps\\utils.py'>

In [2]:
data = pd.read_csv('./data/final_data.csv',parse_dates=['dtcol'])

In [3]:
lags = 3
n_traps = 3

In [4]:
"""counter = 0
for trap in data['narmad'].unique():
    sorted_vector = data[data['narmad'] == trap]['dtcol'].sort_values().reset_index(drop=True) #sort values
    armad_counter = sorted_vector.diff().value_counts()
    counter = armad_counter.add( counter,fill_value=0)

print('Number of samples with certain time difference between the previous one:',)
display(pd.DataFrame(counter.sort_values(ascending=False)[:20].reset_index()).rename(columns={'dtcol':'time_diff'}))"""

"counter = 0\nfor trap in data['narmad'].unique():\n    sorted_vector = data[data['narmad'] == trap]['dtcol'].sort_values().reset_index(drop=True) #sort values\n    armad_counter = sorted_vector.diff().value_counts()\n    counter = armad_counter.add( counter,fill_value=0)\n\nprint('Number of samples with certain time difference between the previous one:',)\ndisplay(pd.DataFrame(counter.sort_values(ascending=False)[:20].reset_index()).rename(columns={'dtcol':'time_diff'}))"

### Distance Matrix

In [81]:
#TODO move to data exploration of preprocessing

same_position_armad = data.groupby(['latitude','longitude'])['narmad'].nunique()
same_position_armad.value_counts() #number of traps in the same position
data['narmad'] = data.groupby(['latitude','longitude'])['narmad'].transform('min') #group by position and take the minimum trap number

In [ ]:
position_matrix = data[['latitude','longitude','narmad']].drop_duplicates().dropna().reset_index(drop=True)

try: 
    distance_matrix = pd.read_csv('./results/distance_matrix.csv')
    distance_matrix.columns = distance_matrix.columns.astype(float)
    distance_matrix.set_index(distance_matrix.columns,inplace=True)

except:
    distance_matrix = utils.create_distance_matrix(position_matrix)
    distance_matrix.to_csv('./results/distance_matrix.csv')

distance_matrix_np = distance_matrix.to_numpy()


### Week trap matrix

In [6]:
week_trap_df = data.pivot(index=['ano','semepi'],columns='narmad',values='novos')
new_index = pd.MultiIndex.from_product([week_trap_df.index.levels[0], range(101,153)]) # introduce weeks 51 and 52
new_index = new_index[(new_index <= week_trap_df.iloc[-1].name) & (new_index >= week_trap_df.iloc[0].name)] #remove indexes that are greater than the last sample or smaller than the first one
week_trap_df = week_trap_df.reindex(new_index) # [week,trap] - > novos


### Lagged Matrix

In [7]:
list_lagged_df = []
for i in range(1,2*lags+1):
    list_lagged_df.append(week_trap_df.shift(i).values)
lagged_matrix = np.stack(list_lagged_df, axis=0) # numpy array with the lagged values. [lag x week x trap] -> novos. Obs.: consider 2*lags due to the biweekly sampling rate


### NaN Count Matrix

In [8]:
nan_count_matrix = np.sum(np.isnan(lagged_matrix), axis=0) # [week x trap] -> number of nans in the lagged matrix

for i in range(2*lags):
    print("Number of samples with",i+1,"valid values: ",np.sum(nan_count_matrix==i+1))

Number of samples with 1 valid values:  0
Number of samples with 2 valid values:  0
Number of samples with 3 valid values:  820386
Number of samples with 4 valid values:  295751
Number of samples with 5 valid values:  33236
Number of samples with 6 valid values:  204019


### Info Matrix

In [9]:
info_df = data[['ano','semepi','nplaca','dtcol','novos','narmad','latitude','longitude']]

### Lagged days matrix

In [10]:
day_df = data.pivot(index=['ano','semepi'],columns='narmad',values='dtcol')
new_index = pd.MultiIndex.from_product([day_df.index.levels[0], range(101,153)]) # introduce weeks 51 and 52
new_index = new_index[(new_index <= day_df.iloc[-1].name) & (new_index >= day_df.iloc[0].name)] #remove indexes that are greater than the last sample or smaller than the first one
day_df = day_df.reindex(new_index) # [week,trap] - > dtcol
day_df = day_df.map(lambda x: x.toordinal() if pd.notnull(x) else np.nan) # convert to ordinal so we can calculate the difference between two dates

list_lagged_days = []
for i in range(1,2*lags+1):
    list_lagged_days.append(day_df.shift(i).values)
lagged_days = np.stack(list_lagged_days, axis=0) # numpy array with the lagged values. [lag x week x trap] -> ordinal daus. Obs.: consider 2*lags due to the biweekly sampling rate




### Useful dicts

In [32]:
trap_index_dict = {trap: index for index,trap in enumerate(distance_matrix.columns)}                                           # trap: index 
yearweek_index_dict = {(year,week): index for index,(year,week) in enumerate(week_trap_df.index)}                           # (year,week): index
nplaca_week_dict = {nplaca: (year, week) for nplaca,week,year in zip(info_df['nplaca'],info_df['semepi'],info_df['ano'])}   # nplaca: (year,week)
nplaca_index_dict = {nplaca: yearweek_index_dict[(year, week)] for nplaca,week,year in zip(info_df['nplaca'],info_df['semepi'],info_df['ano'])}   # nplaca: week index 


### Final Dataframe

In [39]:
for trap in distance_matrix.columns:
    matching_placas = info_df[info_df['narmad'] == trap]['nplaca']
    trap_index = trap_index_dict[trap]
    sorted_distance_indexes = np.argsort(distance_matrix_np[trap_index])
    assert sorted_distance_indexes[0] == trap_index
    

    for placa in matching_placas: 
        pass

AssertionError: 

In [63]:
utils.row_with_value(info_df,'narmad',trap)

,ano,semepi,nplaca,dtcol,novos,narmad,latitude,longitude
70622,2014.0,140.0,90071376.0,2014-10-06,0.0,902026.0,-19.955907,-43.912858
70623,2018.0,118.0,90071594.0,2018-05-09,0.0,902026.0,-19.955907,-43.912858
70624,2017.0,136.0,90071516.0,2017-09-11,0.0,902026.0,-19.955907,-43.912858
70625,2021.0,134.0,90071348.0,2021-08-30,0.0,902026.0,-19.955907,-43.912858
70626,2015.0,104.0,90071460.0,2015-02-02,0.0,902026.0,-19.955907,-43.912858
...,...,...,...,...,...,...,...,...
70922,2017.0,126.0,90071434.0,2017-07-03,0.0,902026.0,-19.955907,-43.912858
70923,2016.0,112.0,90071535.0,2016-03-28,0.0,902026.0,-19.955907,-43.912858
70924,2012.0,106.0,90071385.0,2012-02-13,0.0,902026.0,-19.955907,-43.912858
70925,2021.0,118.0,90071561.0,2021-05-10,0.0,902026.0,-19.955907,-43.912858


In [58]:
info_df.groupby('narmad').min()

,ano,semepi,nplaca,dtcol,novos,latitude,longitude
narmad,,,,,,,
901001,2012.0,101,90000001,2012-05-14,0.0,-19.974521,-44.057499
901002,2012.0,101,90000285,2012-09-03,0.0,-19.984561,-44.053943
901003,2012.0,101,90000568,2012-09-03,0.0,-19.977669,-44.053692
901004,2012.0,101,90000851,2012-09-03,0.0,-19.974816,-44.061480
901005,2012.0,101,90001134,2012-09-03,0.0,-19.977715,-44.057558
...,...,...,...,...,...,...,...
909190,2023.0,102,90524298,2023-11-21,0.0,-19.796700,-43.947889
909191,2023.0,102,90524317,2023-11-21,0.0,-19.786108,-43.971220
909192,2023.0,102,90524336,2023-12-06,0.0,-19.782779,-43.975511


In [61]:
distance_matrix.iloc[242].name

np.float64(902025.0)

In [48]:
sorted_distance_indexes[:10]

array([242, 243, 248, 393, 245, 244, 247, 241, 391, 388])